In [1]:
"""This module implements data feeding and training loop to create model
to classify X-Ray chest images as a lab example for BSU students.
"""

__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""


import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler


LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 64
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)

def build_model():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
						classes = NUM_CLASSES,
                                               include_top=False,
                                               weights='imagenet')
    #base_model.trainable = False
    return tf.keras.models.Sequential([
    	base_model,
    	tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax)
    ])

def main():
    path_train= "/content/drive/My Drive/TFRecords/train*"
    path_val= "/content/drive/My Drive/TFRecords/val*"
 
    train_dataset = create_dataset(glob.glob(path_train), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(path_val), BATCH_SIZE)

    model = build_model()

    model.compile(
        optimizer=tf.optimizers.SGD(lr=0.0000052, momentum=0.9),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    log_dir='/content/drive/My Drive/{}/lab3-5-1/ilcd-{}'.format(LOG_DIR, time.time())
    model.fit(
        train_dataset,
        epochs=150,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
        ]
    )


if __name__ == '__main__':
    main()

9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/150
      1/Unknown - 0s 991us/step - loss: 4.0572 - categorical_accuracy: 0.0312WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
      2/Unknown - 1s 389ms/step - loss: 3.9153 - categorical_accuracy: 0.0312WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1182s vs `on_train_batch_end` time: 0.6562s). Check your callbacks.
220/220 [==============================] - 100s 456ms/step - loss: 2.3431 - categorical_accuracy: 0.3278 - val_loss: 3.1333 - val_categorical_accuracy: 0.1913
Epoch 2/150
220/220 [==============================] - 92s 420ms/step - loss: 2.1156 - categorical_accuracy: 0.3370 - val_loss: 2.8095 - val_categorical_accura